In [1]:
import requests
import numpy as np
import pandas as pd 
import json
import folium
import warnings
from tqdm import tqdm_notebook
from urllib.parse import urlparse
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from glob import glob 
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
key_fd = open('kakaomapkey.txt', mode='r')
kmap_key = key_fd.read(100)
key_fd.close()

In [3]:
station_files = glob('./station_data/지역*.xls')
tmp_raw = []

for file_name in station_files:
    tmp = pd.read_excel(file_name, header=2)
    tmp_raw.append(tmp)

station_raw = pd.concat(tmp_raw)

In [4]:
stations = pd.DataFrame({
    'Oil_store': station_raw['상호'],
    '주소': station_raw['주소'],
    '가격': station_raw['휘발유'],
    '셀프': station_raw['셀프여부'],
    '상표': station_raw['상표']
})
stations.head()

,Oil_store,주소,가격,셀프,상표
0,구천면주유소,서울 강동구 구천면로 357 (암사동),1374,N,현대오일뱅크
1,지에스칼텍스㈜ 신월주유소,서울 강동구 양재대로 1323 (성내동),1385,N,GS칼텍스
2,현대오일뱅크㈜직영 고속셀프주유소,서울 강동구 천호대로 1246,1386,Y,현대오일뱅크
3,현대오일뱅크㈜직영 명일셀프주유소,서울 강동구 고덕로 168 (명일동),1395,Y,현대오일뱅크
4,방아다리주유소,서울 강동구 동남로 811 (명일동),1395,Y,SK에너지


In [5]:
stations['구'] = [addr.split()[1] for addr in stations['주소']]

In [6]:
stations = stations[stations['가격'] != '-']

In [7]:
stations['가격'] = pd.to_numeric(stations['가격'])

In [8]:
stations.tail()

,Oil_store,주소,가격,셀프,상표,구
33,갤러리아주유소,서울 강남구 압구정로 426,1848,N,SK에너지,강남구
34,학여울주유소,서울 강남구 남부순환로 3170 (일원2동),1858,N,GS칼텍스,강남구
35,(주)소모에너지엔테크놀러지 쎈트럴주유소,서울 강남구 삼성로 335,1883,N,GS칼텍스,강남구
36,(주)만정에너지,서울 강남구 봉은사로 433 (삼성동),1913,N,GS칼텍스,강남구
37,뉴서울(강남),서울 강남구 언주로 716 (논현동),2025,N,SK에너지,강남구


In [9]:
stations.reset_index(drop=True, inplace=True)
stations

,Oil_store,주소,가격,셀프,상표,구
0,구천면주유소,서울 강동구 구천면로 357 (암사동),1374,N,현대오일뱅크,강동구
1,지에스칼텍스㈜ 신월주유소,서울 강동구 양재대로 1323 (성내동),1385,N,GS칼텍스,강동구
2,현대오일뱅크㈜직영 고속셀프주유소,서울 강동구 천호대로 1246,1386,Y,현대오일뱅크,강동구
3,현대오일뱅크㈜직영 명일셀프주유소,서울 강동구 고덕로 168 (명일동),1395,Y,현대오일뱅크,강동구
4,방아다리주유소,서울 강동구 동남로 811 (명일동),1395,Y,SK에너지,강동구
...,...,...,...,...,...,...
482,갤러리아주유소,서울 강남구 압구정로 426,1848,N,SK에너지,강남구
483,학여울주유소,서울 강남구 남부순환로 3170 (일원2동),1858,N,GS칼텍스,강남구
484,(주)소모에너지엔테크놀러지 쎈트럴주유소,서울 강남구 삼성로 335,1883,N,GS칼텍스,강남구
485,(주)만정에너지,서울 강남구 봉은사로 433 (삼성동),1913,N,GS칼텍스,강남구


In [10]:
stations.iloc[50]

Oil_store             용강주유소
주소           서울 마포구 토정로 258
가격                     1386
셀프                        N
상표                    S-OIL
구                       마포구
Name: 50, dtype: object

In [11]:
stations.index

RangeIndex(start=0, stop=487, step=1)

In [12]:
lat_list = []; lng_list = []
for i in tqdm_notebook(stations.index):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + stations['주소'][i]
    result = requests.get(urlparse(url).geturl(), headers={"Authorization":"KakaoAK " + kmap_key}).json()
    if result['documents'] == []:
        lat_list.append(np.nan); lng_list.append(np.nan)
    else:
        lat = float(result['documents'][0]['y'])
        lng = float(result['documents'][0]['x'])
        lat_list.append(lat); lng_list.append(lng)

In [13]:
len(stations['Oil_store']), len(stations['주소']), len(stations['가격']), len(stations['셀프']), len(stations['상표']), len(lat_list), len(lng_list)

(487, 487, 487, 487, 487, 487, 487)

In [14]:
stations2 = pd.DataFrame({
    'Oil_store': stations['Oil_store'],
    '주소': stations['주소'],
    '가격': stations['가격'],
    '셀프': stations['셀프'],
    '상표': stations['상표'],
    '위도': lat_list,
    '경도': lng_list
})
stations2.head()

,Oil_store,주소,가격,셀프,상표,위도,경도
0,구천면주유소,서울 강동구 구천면로 357 (암사동),1374,N,현대오일뱅크,37.550458,127.138227
1,지에스칼텍스㈜ 신월주유소,서울 강동구 양재대로 1323 (성내동),1385,N,GS칼텍스,37.524535,127.134421
2,현대오일뱅크㈜직영 고속셀프주유소,서울 강동구 천호대로 1246,1386,Y,현대오일뱅크,37.536417,127.149372
3,현대오일뱅크㈜직영 명일셀프주유소,서울 강동구 고덕로 168 (명일동),1395,Y,현대오일뱅크,37.554479,127.144011
4,방아다리주유소,서울 강동구 동남로 811 (명일동),1395,Y,SK에너지,37.546061,127.155495


In [15]:
oil_price_top10 = stations2.sort_values(by='가격', ascending=False).head(10)
oil_price_bottom10 = stations2.sort_values(by='가격', ascending=True).head(10)
oil_price_top10.reset_index(drop=True, inplace=True)
oil_price_bottom10.reset_index(drop=True, inplace=True)
oil_price_top10.head()

,Oil_store,주소,가격,셀프,상표,위도,경도
0,서남주유소,서울 중구 통일로 30,2183,N,SK에너지,37.558384,126.972172
1,서계주유소,서울 용산구 청파로 367 (서계동),2089,N,GS칼텍스,37.552366,126.968994
2,필동주유소,서울 중구 퇴계로 196 (필동2가),2079,N,GS칼텍스,37.560890,126.993676
3,SK에너지(주) 퇴계로주유소,서울 중구 퇴계로 228 (필동2가),2069,N,SK에너지,37.561709,126.997141
4,뉴서울(강남),서울 강남구 언주로 716 (논현동),2025,N,SK에너지,37.517669,127.035823


In [16]:
geo_path = './station_data/skorea_municipalities_geo_simple.json'
geo_data = json.load(open(geo_path, encoding='utf-8'))
gu_data = stations['가격'].groupby(stations['구']).mean()

In [17]:
# 휘발유 가격이 가장 비싼 주유소 Top 10 위치 표시
map = folium.Map(location=[stations2['위도'].mean(), stations2['경도'].mean()], zoom_start=10.5)
map.choropleth(geo_data=geo_data,
                data=gu_data,
                columns=[gu_data.index, '가격'],
                fill_color='PuRd',
                key_on='feature.id')

for i in oil_price_top10.index:
    folium.Marker(
        location=[oil_price_top10['위도'][i], oil_price_top10['경도'][i]],
        popup=oil_price_top10['주소'][i],
        tooltip=f"휘발유 가격: {oil_price_top10['가격'][i]}원",
        icon=folium.Icon(color='red', icon='tag')
    ).add_to(map)
map

In [18]:
# 휘발유 가격이 가장 저렴한 주유소 Top 10 위치 표시
map = folium.Map(location=[stations2['위도'].mean(), stations2['경도'].mean()], zoom_start=10.5)
map.choropleth(geo_data=geo_data,
                data=gu_data,
                columns=[gu_data.index, '가격'],
                fill_color='PuRd',
                key_on='feature.id')

for i in oil_price_bottom10.index:
    folium.Marker(
        location=[oil_price_bottom10['위도'][i], oil_price_bottom10['경도'][i]],
        popup=oil_price_bottom10['주소'][i], 
        tooltip=f"휘발유 가격: {oil_price_bottom10['가격'][i]}원",
        icon=folium.Icon(color='blue', icon='map-marker')
    ).add_to(map)
map

In [19]:
# 휘발유 가격이 가장 비싼 10곳과 가장 저렴한 10곳을 같이 표시
map = folium.Map(location=[stations2['위도'].mean(), stations2['경도'].mean()], zoom_start=10.5)
map.choropleth(geo_data=geo_data,
                data=gu_data,
                columns=[gu_data.index, '가격'],
                fill_color='PuRd',
                key_on='feature.id')

for i in range(10):
    folium.Marker(
        location=[oil_price_top10['위도'][i], oil_price_top10['경도'][i]],
        popup=oil_price_top10['주소'][i],
        tooltip=f"휘발유 가격: {oil_price_top10['가격'][i]}원",
        icon=folium.Icon(color='red', icon='tag')
    ).add_to(map)

    folium.Marker(
        location=[oil_price_bottom10['위도'][i], oil_price_bottom10['경도'][i]],
        popup=oil_price_bottom10['주소'][i], 
        tooltip=f"휘발유 가격: {oil_price_bottom10['가격'][i]}원",
        icon=folium.Icon(color='blue', icon='map-marker')
    ).add_to(map)
map